# Demo Long Context Lengths

- Prompt 128K question to Llama 3.3 11B which has 128K context length
- Prompt 128K question to DeepSeek-R1 model which has 8K context length - error!
- Naive Recursive Text Splitting + 1 extra combining prompt to DeepSeek-R1
- Prompt distilled DeepSeek-R1 model with 128K context length
- Prompt YaRn (new RoPE) fine tuned 32K model on vLLM

In [1]:
# SambaNova chat uses OpenAI API
# https://github.com/sambanova/ai-starter-kit/blob/main/quickstart/README.md
# from openai import OpenAI
import os, tiktoken, time, pprint
from IPython.display import display, Markdown

# SambaNova LangChain integration to chain structured output
# !python3 -m pip install langchain-sambanova
from langchain_sambanova import ChatSambaNovaCloud

# Authenticate into SambaNova Cloud using API key
api_key = os.environ.get("SAMBANOVA_API_KEY")

# Chat mode
streaming = False  #streaming turned off

# List all models
# https://cloud.sambanova.ai > Playground
model="Meta-Llama-3.3-70B-Instruct"

# Wrap LangChain model for SambaNova
llm = ChatSambaNovaCloud(
    model=model,
    max_tokens=1024,
    temperature=0.7,
    top_p=0.01,
)

/Users/christy/Documents/py312/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in SambaStudioEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [4]:
from pydantic import BaseModel, Field
from typing import List, Dict

# topic_desc = """ 
# Concise summary of the key contextual difference between Doc1 and Doc2 for the specific topic. 
# Focus on the meaningful distinction and its practical implications.
# """

# doc1_context_desc = """Relevant text excerpts from Doc1 that pertain to the topic. 
# Within this quoted text, use bold markdown formatting to highlight
# or phrases that are different or absent compared to the corresponding clauses in Doc2.
# """

# doc2_context_desc = """Corresponding text excerpts from Doc2 that address the same topic. 
# Within this quoted text, use bold markdown formatting to highlight
# or phrases that are different or absent compared to the clause in Doc1.
# """

# Pydantic class that represents a single difference between 2 docs.
# Use OpenAI tool schema (JSON)
class Difference(BaseModel):
    topic: str = Field(..., description="Topic")
    summary: str = Field(..., description="Summary of differences per topic")
    doc1_context: str = Field(..., description="Context sentences from Document 1 with all and only differing words highlighted.")
    doc2_context: str = Field(..., description="Context sentences from Document 2 with all and only differing words highlighted.")

# Pydantic class containing a list of Difference objects.
class Differences(BaseModel):
    differences: List[Difference] = Field(..., description="A list of differences between two documents.")

# Function to count tokens
def count_openai_tokens(prompt):
    # Count how many OpenAI tokens are used by the prompt
    encoding = tiktoken.get_encoding("cl100k_base")

    # Encode the prompt to get the tokens
    tokens = encoding.encode(prompt)

    # Count the number of tokens
    num_tokens = len(tokens)

    # Print the number of tokens
    print(f"Number of tokens in the prompt: {num_tokens}")

    return num_tokens

# Function to display markdown of JSON "differences"
def display_differences_as_markdown(differences):
    """
    Function to display differences in a markdown table format.
    
    Args:
        differences (Differences): An instance of the Differences class containing a list of Difference objects.
    """
    # Start the markdown table
    markdown_table = "| Topic | Summary Differences | Doc1 Context | Doc2 Context |\n"
    markdown_table += "|-------|--------------|--------------|--------------\n"
    
    # Loop through each difference and format it into the table
    for diff in differences.differences:
        markdown_table += f"| {diff.topic} | {diff.summary} | {diff.doc1_context} | {diff.doc2_context} |\n"
    
    # Display the markdown table
    display(Markdown(markdown_table))

# Example documents

- Prompt token count = 10367  (10K prompt for 8K context window)
- Replace doc1, with all text from https://sambanova.ai/cloud-end-user-license-agreement
- Replace doc2, with all text from https://developers.google.com/terms

In [3]:
# Define the path to the data folder
data_folder = 'data'

# Read the contents of doc1.txt and doc2.txt
with open(os.path.join(data_folder, 'doc1.txt'), 'r', encoding='utf-8') as file:
    doc1 = file.read()

with open(os.path.join(data_folder, 'doc2.txt'), 'r', encoding='utf-8') as file:
    doc2 = file.read()

In [6]:
# doc1 = "The quick brown fox jumped over the lazy dog"
# doc2 = "The slow brown fox jumped over the lazy cat"

prompt_template = """You are a legal analyst tasked with performing a detailed comparative analysis of two lengthy legal agreements:
Doc1: {doc1}
Doc2: {doc2}

Analyze what each document says about each topic carefully to identify and present the contextual differences 
in how each document addresses each topic. The topics to compare are:
- Definition of Confidential Information
- Permitted Use & Restrictions
- Data Security

Output JSON summarizing the contextual differences between Doc1 and Doc2 for each topic.
JSON keys: 
topic, summary, doc1_context, doc2_context

summary: 
Provide a concise summary of the key contextual difference between Doc1 and Doc2 
for the specific feature. Focus on the *meaningful distinction* and its *practical implications*.

doc1_context: 
Quote the *relevant text excerpt* from Doc1 that pertains to the feature. 
**Within this quoted text, use bold markdown formatting to highlight the specific words 
or phrases that are different or absent compared to the corresponding clause in Doc2.**

doc2_context: 
Quote the *corresponding text excerpt* from Doc2 that addresses the same feature. 
**Within this quoted text, use bold markdown formatting to highlight the specific words 
or phrases that are different or absent compared to the clause in Doc1.**

Use bold markdown to highlight the differing text within the "doc1_context" and "doc2_context" columns as described above.
Make sure you highlight in bold for each row, only text differences, to make it easier for the user to see the differences.
"""

# Assemble the prompt
prompt = prompt_template.format(
    doc1=doc1,
    doc2=doc2)

# Print number of tokens in the prompt
num_tokens = count_openai_tokens(prompt)

Number of tokens in the prompt: 10057


# Meta-Llama-3.3-70B-Instruct
128K Context Length

In [7]:
# Chat completion using the specified model.

# List all models
# https://cloud.sambanova.ai > Playground
model="Meta-Llama-3.3-70B-Instruct"

# Wrap LangChain model for SambaNova
llm = ChatSambaNovaCloud(
    model=model,
    max_tokens=1024,
    temperature=0.7,
    top_p=0.01,
)

start_time = time.time()  # Start timing

# Chat with SambaNova chat completion chain to structured output.
# Ask LLM to return structured output conforming to Differences Pydantic class.
# That is, return a list of JSON Difference elements.
structured_llm = llm.with_structured_output(Differences)

# Invoke the model-to-structured-output chain using a prompt.
completion = structured_llm.invoke(prompt)

end_time = time.time()  # End timing
response_time = end_time - start_time  # Calculate response time
print(f"Response time: {response_time} seconds for model: {model}")

# # Loop through list of Difference JSON elements.
# for diff in completion.differences:
#     pprint.pprint(f"topic: {diff.topic}")
#     pprint.pprint(f"Doc1: {diff.doc1_context}")
#     pprint.pprint(f"Doc2: {diff.doc2_context}") 

# Render the structured completion JSON as markdown table
display_differences_as_markdown(completion)

Response time: 3.4840099811553955 seconds for model: Meta-Llama-3.3-70B-Instruct


| Topic | Summary Differences | Doc1 Context | Doc2 Context |
|-------|--------------|--------------|--------------
| Definition of Confidential Information | Doc1 defines Confidential Information as all information that is identified as confidential at the time of disclosure or should be reasonably known to be confidential or proprietary due to the nature of the information disclosed and the circumstances surrounding the disclosure. In contrast, Doc2 does not provide a clear definition of Confidential Information, but rather refers to it as 'confidential matters' and describes the obligations related to it. | **Confidential Information** shall mean all information that is identified as confidential at the time of disclosure or should be reasonably known to be confidential or proprietary due to the nature of the information disclosed and the circumstances surrounding the disclosure. | Our communications to you and our APIs may contain **Google confidential information**. Google confidential information includes any materials, communications, and information that are marked confidential or that would normally be considered confidential under the circumstances. |
| Permitted Use & Restrictions | Doc1 grants a non-exclusive, non-transferable, limited license to use the software embodied in the Service, while Doc2 allows use of the APIs, but with specific restrictions and requirements, such as complying with applicable law and not using the APIs to encourage or promote illegal activity. | Subject to and conditioned upon Customer’s and its Users’ strict compliance with all terms and conditions set forth in this Agreement and the Additional Documents, SambaNova hereby grants to Customer a **non-exclusive, non-transferable, limited license** during the Term to use the software embodied in the Service. | You will comply with all applicable law, regulation, and third party rights (including without limitation laws regarding the import or export of data or software, privacy, and local laws). You will not use the APIs to **encourage or promote illegal activity or violation of third party rights**. |
| Data Security | Doc1 requires the implementation and use of reasonable and appropriate measures to help secure the Service, while Doc2 requires the use of commercially reasonable efforts to protect user information collected by the API Client. | You must implement and use **reasonable and appropriate measures** to help secure the Service by you and your Users. | You will use **commercially reasonable efforts** to protect user information collected by your API Client, including personal data, from unauthorized access or use. |


In [8]:
print(completion)  # for copy/paste to eval table

differences=[Difference(topic='Definition of Confidential Information', summary="Doc1 defines Confidential Information as all information that is identified as confidential at the time of disclosure or should be reasonably known to be confidential or proprietary due to the nature of the information disclosed and the circumstances surrounding the disclosure. In contrast, Doc2 does not provide a clear definition of Confidential Information, but rather refers to it as 'confidential matters' and describes the obligations related to it.", doc1_context='**Confidential Information** shall mean all information that is identified as confidential at the time of disclosure or should be reasonably known to be confidential or proprietary due to the nature of the information disclosed and the circumstances surrounding the disclosure.', doc2_context='Our communications to you and our APIs may contain **Google confidential information**. Google confidential information includes any materials, communic

# DeepSeek-R1
8K Context Length

The example input docs have token length=10540. <br>
If you just submit the chat completion, you'll get error message: <br>

`BadRequestError: The maximum context length of DeepSeek-R1 is 8192. However, answering your request will take 10541 tokens. Please reduce the length of the messages.`

In [9]:
# Chat completion using the specified model.

# List all models
# https://cloud.sambanova.ai > Playground
model = "DeepSeek-R1"

# Wrap LangChain model for SambaNova
llm = ChatSambaNovaCloud(
    model=model,
    max_tokens=1024,
    temperature=0.7,
    top_p=0.01,
)

# start_time = time.time()  # Start timing

# # Chat with SambaNova chat completion chain to structured output.
# structured_llm = llm.with_structured_output(Differences)

# # Invoke the model-to-structured-output chain using a prompt.
# completion = structured_llm.invoke(prompt)

# end_time = time.time()  # End timing
# response_time = end_time - start_time  # Calculate response time
# print(f"Response time: {response_time} seconds for model: {model}")

# failed with status code 400.', 'The maximum context length of DeepSeek-R1 is 8192. 
# However, answering your request will take 11082 tokens. Please reduce the length of 
# the messages or the specified max_completion_tokens value.

RuntimeError: ('Sambanova /complete call failed with status code 400.', 'The maximum context length of DeepSeek-R1 is 8192. However, answering your request will take 11082 tokens. Please reduce the length of the messages or the specified max_completion_tokens value.\n.')

# Chunk texts before sending prompts to DeepSeek-R1

In [51]:
from IPython.display import display, Markdown

# Extract json from chat completion
def extract_and_display_output(completion,
                               structured_output_start = "```json\n[\n  ", 
                               print_output=True):

    try:
        # Assuming `completion` is the response from the OpenAI API
        diff_dict = completion.__dict__
        # Extract answer from the response
        response_content = diff_dict['choices'][0].message.content
    except:
        response_content = completion

    # Extract structured part only from the answer
    if structured_output_start in response_content:
        # Extract the part of the answer starting from the structured output
        structured_output = response_content.split(structured_output_start)[1].strip()

        # Remove trailing extra characters
        structured_output = structured_output.split(']\n')[0].strip()

        # Add beginning and ending []
        structured_output = "[" + structured_output + "]"
    else:
        structured_output = response_content  # Fallback if the tag is not found"
    # debugging print
    print(structured_output)

    # Return raw markdown text
    return structured_output

# Display json as a markdown table
def display_json_as_markdown(json_string):
    """
    Function to display JSON data in a markdown table format.
    
    Args:
        json_string (str): A JSON string containing the data to display.
    """
    # Load the JSON string into a Python object
    try:
        json_data = json.loads(json_string)
    except json.JSONDecodeError:
        print("Invalid JSON string provided.")
        return

    # Check if json_data is not empty
    if not json_data:
        print("No data to display.")
        return

    # Extract headers from the first dictionary
    headers = json_data[0].keys()
    
    # Start the markdown table
    markdown_table = "| " + " | ".join(headers) + " |\n"
    markdown_table += "| " + " | ".join(["---"] * len(headers)) + " |\n"
    
    # Loop through each dictionary and format it into the table
    for entry in json_data:
        row = " | ".join(str(entry[header]) for header in headers)
        markdown_table += f"| {row} |\n"
    
    # Display the markdown table
    display(Markdown(markdown_table))

In [ ]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# # CHUNK_SIZE = 1000
# CHUNK_SIZE = 2000
# # CHUNK_SIZE = 3500

# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
#     # model_name="gpt-3.5-turbo",
#     chunk_size=CHUNK_SIZE,
#     chunk_overlap=150
# )

# # Split the documents into chunks
# doc1_chunks = text_splitter.split_text(doc1)
# doc2_chunks = text_splitter.split_text(doc2)

# # Display the number of chunks created
# print(f"Number of chunks in Document 1: {len(doc1_chunks)}")
# print(f"Number of chunks in Document 2: {len(doc2_chunks)}")

# # Prepare the prompt for each chunk and collect responses
# start_time = time.time()  # Start timing
# responses = []
# for i in range(max(len(doc1_chunks), len(doc2_chunks))):
#     chunk1 = doc1_chunks[i] if i < len(doc1_chunks) else ""
#     chunk2 = doc2_chunks[i] if i < len(doc2_chunks) else ""
    
#     prompt_chunk = prompt_template.format(doc1=chunk1, doc2=chunk2)
#     num_tokens = count_openai_tokens(prompt=prompt_chunk)

#     # Chat with SambaNova chat completion chain to structured output.
#     structured_llm = llm.with_structured_output(Differences)

#     # Invoke the model-to-structured-output chain using a prompt.
#     completion = structured_llm.invoke(prompt_chunk)

#     # Collect the response
#     responses.append(completion.choices[0].message.content)

# end_time = time.time()  # End timing
# response_time_chunks = end_time - start_time  # Calculate response time
# print(f"Time to process chunks: {response_time_chunks}")

# failed with status code 200.', "{'error': {'code': None, 'message': 
# 'Model DeepSeek-R1 does not support tool use.'

In [56]:
# SambaNova native API uses OpenAI API
from openai import OpenAI
# https://github.com/sambanova/ai-starter-kit/blob/main/quickstart/README.md

# Authenticate into SambaNova Cloud using API key
api_key = os.environ.get("SAMBANOVA_API_KEY")
client = OpenAI(
    base_url="https://api.sambanova.ai/v1/",
    api_key=api_key,  
)

from langchain.text_splitter import RecursiveCharacterTextSplitter

# CHUNK_SIZE = 1000
# CHUNK_SIZE = 2000
CHUNK_SIZE = 3500

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    # model_name="gpt-3.5-turbo",
    chunk_size=CHUNK_SIZE,
    chunk_overlap=150
)

# Split the documents into chunks
doc1_chunks = text_splitter.split_text(doc1)
doc2_chunks = text_splitter.split_text(doc2)

# Display the number of chunks created
print(f"Number of chunks in Document 1: {len(doc1_chunks)}")
print(f"Number of chunks in Document 2: {len(doc2_chunks)}")

# Prepare the prompt for each chunk and collect responses
start_time = time.time()  # Start timing
responses = []
for i in range(max(len(doc1_chunks), len(doc2_chunks))):
    chunk1 = doc1_chunks[i] if i < len(doc1_chunks) else ""
    chunk2 = doc2_chunks[i] if i < len(doc2_chunks) else ""
    
    prompt_chunk = prompt_template.format(doc1=chunk1, doc2=chunk2)
    num_tokens = count_openai_tokens(prompt=prompt_chunk)

    # Call the OpenAI API to get the response for each chunk
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user", 
                "content": prompt_chunk,
            }
        ],
        stream=streaming,
    )
    # Collect the response
    responses.append(completion.choices[0].message.content)

end_time = time.time()  # End timing
response_time_chunks = end_time - start_time  # Calculate response time
print(f"Time to process chunks: {response_time_chunks} seconds")

# 37.26542592048645 seconds

Number of chunks in Document 1: 2
Number of chunks in Document 2: 2
Number of tokens in the prompt: 6825
Number of tokens in the prompt: 3740
Time to process chunks: 10.227445840835571 seconds


In [52]:
# # Loop through list of responses.
# i = 0
# for diff in responses:
#     pprint.pprint(f"CHUNK {i}: {diff}")
#     print()
#     i += 1

### Naive chunking approach needs 1 more prompt to combine chunks into single answer

In [18]:
# Given chunk responses, combine into 1 single table

# Assemble the prompt
prompt_combine = f""" You are a highly skilled legal document analyst. 
Your task is to synthesize key contextual differences between 2 legal documents into a single, structured markdown table. 

Given an input string containing multiple separate chunk answers. 
Each chunk represents separate analysis for Doc1 and Doc2, related to different legal topics.
Input string: {responses}

**Objective:** 
Combine the information from these multiple input analyses into **one cohesive markdown table** with **only three rows**. 
This single output table will highlight the contextual differences between Doc1 and Doc2 across key features. 

**Instructions:** 
1. **Understand the differences in each chunk:**  

2. **Focus on Key Topics:**  Your comparison should be centered around three critical topics common in the chunks: 
- Definition of Confidential Information
- Permitted Use & Restrictions
- Data Security

3. **Analyze and Identify Contextual Differences:** 
 Carefully analyze the clauses from Doc1 and Doc2 related to each of the topics above. 
 Your goal is to identify and articulate the *contextual differences* in how each document addresses these areas.  
 Contextual differences refer to meaningful distinctions in the language, scope, or implications of the clauses in each document. 

Output JSON summarizing the contextual differences between Doc1 and Doc2 for each topic.
JSON keys: 
topic, summary, doc1_context, doc2_context

summary: 
Provide a concise summary of the key contextual difference between Doc1 and Doc2 
for the specific feature. Focus on the *meaningful distinction* and its *practical implications*.

doc1_context: 
Quote the *relevant text excerpt* from Doc1 that pertains to the feature. 
**Within this quoted text, use bold markdown formatting to highlight the specific words 
or phrases that are different or absent compared to the corresponding clause in Doc2.**

doc2_context: 
Quote the *corresponding text excerpt* from Doc2 that addresses the same feature. 
**Within this quoted text, use bold markdown formatting to highlight the specific words 
or phrases that are different or absent compared to the clause in Doc1.**

Use bold markdown to highlight the differing text within the "doc1_context" and "doc2_context" columns as described above.
Make sure you highlight in bold for each row, only text differences, to make it easier for the user to see the differences.
"""

# Print number of tokens in the prompt
num_tokens = count_openai_tokens(prompt_combine)

Number of tokens in the prompt: 4539


In [58]:
# Call the OpenAI API to get the response for each chunk
start_time = time.time()  # Start timing
completion_combine = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "user", 
            "content": prompt_combine,
        }
    ],
    stream=streaming,
)

end_time = time.time()  # End timing
response_time = end_time - start_time  # Calculate response time
print(f"Response time to combine chunks: {response_time} seconds")

# Total response time = time for each chunk + combining chunks together
total_response_time = response_time + response_time_chunks
print(f"Total response time to combine chunks: {total_response_time} seconds")

# Extract just the json structured output from a chat completion
structured_output = extract_and_display_output(completion)

# Display markdown table
display_json_as_markdown(structured_output)

Response time to combine chunks: 2.7475037574768066 seconds
Total response time to combine chunks: 12.974949598312378 seconds
[{
        "topic": "Definition of Confidential Information",
        "summary": "Doc1 explicitly defines Confidential Information, while Doc2 does not provide a specific definition, which could lead to ambiguity in understanding what constitutes confidential information.",
        "doc1_context": "“Confidential Information” shall mean all information that is identified as confidential at the time of disclosure or should be reasonably known to be confidential or proprietary due to the nature of the information disclosed and the circumstances surrounding the disclosure.",
        "doc2_context": "No explicit definition of Confidential Information is provided in Doc2."
    },
    {
        "topic": "Permitted Use & Restrictions",
        "summary": "Doc1 includes export control restrictions, while Doc2 focuses on proper usage and legal compliance without explicit 

| topic | summary | doc1_context | doc2_context |
| --- | --- | --- | --- |
| Definition of Confidential Information | Doc1 explicitly defines Confidential Information, while Doc2 does not provide a specific definition, which could lead to ambiguity in understanding what constitutes confidential information. | “Confidential Information” shall mean all information that is identified as confidential at the time of disclosure or should be reasonably known to be confidential or proprietary due to the nature of the information disclosed and the circumstances surrounding the disclosure. | No explicit definition of Confidential Information is provided in Doc2. |
| Permitted Use & Restrictions | Doc1 includes export control restrictions, while Doc2 focuses on proper usage and legal compliance without explicit permitted use guidelines. | Customer agrees to comply with all export and import laws and regulations of the United States and other applicable jurisdictions. Without limiting the foregoing, (i) Customer represents and warrants that it is not listed on any U.S. government list of prohibited or restricted parties or located in (or a national of) a country that is subject to a U.S. government embargo or that has been designated by the U.S. government as a “terrorist supporting” country, (ii) Customer will not (and will not permit any third parties to) access or use any Service in violation of any U.S. export embargo, prohibition or restriction, and (iii) Customer will not submit to any Service any information that is controlled under the U.S. International Traffic in Arms Regulations. | Unless prohibited by applicable law, if you are a business, you will defend and indemnify Google, and its affiliates, directors, officers, employees, and users, against all liabilities, damages, losses, costs, fees (including legal fees), and expenses relating to any allegation or third-party legal proceeding to the extent arising from: your misuse or your end user's misuse of the APIs; your violation or your end user's violation of the Terms; or any content or data routed into or used with the APIs by you, those acting on your behalf, or your end users. |
| Data Security | Doc1 indirectly addresses data security through hardware liability, while Doc2 does not provide explicit data security measures, leaving it to user responsibility. | NOTHING IN THIS AGREEMENT EXCLUDES OR LIMITS EITHER PARTY’S LIABILITY FOR ... FAILURE TO RETURN, DAMAGE OR LOSS TO SAMBANOVA HARDWARE; | No explicit mention of data security measures is provided in Doc2. |


# GRPO (DeepSeek-R1) fine tuned Llama 3.3 70B
128K Context Length

In [ ]:
# # # Chat completion using the specified model.

# # List all models
# # https://cloud.sambanova.ai > Playground
# model = "DeepSeek-R1-Distill-Llama-70B"

# # Wrap LangChain model for SambaNova
# llm = ChatSambaNovaCloud(
#     model=model,
#     max_tokens=1024,
#     temperature=0.7,
#     top_p=0.01,
# )

# start_time = time.time()  # Start timing

# # Chat with SambaNova chat completion chain to structured output.
# structured_llm = llm.with_structured_output(Differences)

# # Invoke the model-to-structured-output chain using a prompt.
# completion = structured_llm.invoke(prompt)

# end_time = time.time()  # End timing
# response_time = end_time - start_time  # Calculate response time
# print(f"Response time: {response_time} seconds for model: {model}")

# # Render the structured completion JSON as markdown table
# display_differences_as_markdown(completion)

# # failed with status code 200.', "{'error': {'code': None, 'message': 
# # 'Model DeepSeek-R1-Distill-Llama-70B does not support tool use.'

In [53]:
# # Chat completion using the specified model.

# List all models
# https://cloud.sambanova.ai > Playground
model = "DeepSeek-R1-Distill-Llama-70B"

# Print number of tokens in the prompt
num_tokens = count_openai_tokens(prompt)

start_time = time.time()  # Start timing
completion = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "user", 
            "content": prompt, # back to the original 10K length prompt
        }
    ],
    stream=streaming,
)

end_time = time.time()  # End timing
response_time = end_time - start_time  # Calculate response time
print(f"Time to process chunks: {response_time_chunks} seconds")

# Extract just the json structured output from a chat completion
structured_output = extract_and_display_output(completion)

# Display markdown table
display_json_as_markdown(structured_output)

Number of tokens in the prompt: 10057
[{
        "topic": "Definition of Confidential Information",
        "summary": "Doc1 defines Confidential Information broadly, including any information reasonably considered confidential, and emphasizes equitable relief for breaches. Doc2 specifically excludes certain information and requires prior notice for legal disclosures.",
        "doc1_context": "Confidential Information shall mean all information that is identified as confidential at the time of disclosure or should be reasonably known to be confidential or proprietary due to the nature of the information disclosed and the circumstances surrounding the disclosure. The Receiving Party must use the same degree of care as they would for their own confidential information, but not less than reasonable care. They can only use it for the scope of the agreement and must limit access to those who need it. If legally required to disclose, they must notify the Disclosing Party and cooperate to se

| topic | summary | doc1_context | doc2_context |
| --- | --- | --- | --- |
| Definition of Confidential Information | Doc1 defines Confidential Information broadly, including any information reasonably considered confidential, and emphasizes equitable relief for breaches. Doc2 specifically excludes certain information and requires prior notice for legal disclosures. | Confidential Information shall mean all information that is identified as confidential at the time of disclosure or should be reasonably known to be confidential or proprietary due to the nature of the information disclosed and the circumstances surrounding the disclosure. The Receiving Party must use the same degree of care as they would for their own confidential information, but not less than reasonable care. They can only use it for the scope of the agreement and must limit access to those who need it. If legally required to disclose, they must notify the Disclosing Party and cooperate to seek confidential treatment. Disclosing Confidential Information would cause substantial harm, so the Disclosing Party can seek equitable relief. | Confidential Information includes developer credentials and Google's communications marked confidential. It explicitly excludes information independently developed, rightfully obtained from a third party without confidentiality, or that becomes public through no fault of the recipient. The Receiving Party must not disclose it to third parties without prior written consent, unless legally compelled, in which case they must provide prior notice unless a court prohibits it. |
| Permitted Use & Restrictions | Doc1 focuses on internal use and prohibits competing activities, while Doc2 has broader restrictions, especially on content handling and harmful activities. | Allows use solely for Customer's benefit with a non-exclusive license. Prohibits reverse engineering, resale, benchmarking without consent, and competing product development. Requires User compliance. | Permits API use to enhance applications but prohibits sublicensing, misrepresentation, illegal activities, reverse engineering, and harmful uses. Restricts content scraping and caching beyond limits. |
| Data Security | Doc1 shifts compliance responsibility to the Customer, while Doc2 requires active data protection and privacy policy adherence. | Requires reasonable security measures and vulnerability reporting. SambaNova isn't responsible for legal compliance, placing the burden on the Customer. | Mandates commercially reasonable efforts to protect user information, including personal data, with prompt breach reporting. Requires compliance with privacy laws and Google's User Data Policy, including a clear privacy policy. |


#### Try Together.ai

- The [free](https://api.together.xyz/playground/chat/deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free) GRPO fine-tuned model only has 8K context length
- The regular GRPO fine-tuned model has 128K context length


In [55]:
from together import Together

client = Together()

start_time = time.time()  # Start timing

# Using paid GRPO DeepSeek R1 endpoint since it supports longer context length
response = client.chat.completions.create(
    # model="deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free",
    model="deepseek-ai/DeepSeek-R1-Distill-Llama-70B",
    messages=[{"role": "user", "content": prompt}], 
    max_tokens=10200,  # default=None
    temperature=0.7,
    top_p=0.7,
    top_k=50,
    repetition_penalty=1,
    stop=["<｜end▁of▁sentence｜>"],
    stream=False
)
end_time = time.time()  # End timing
response_time = end_time - start_time  # Calculate response time
print(f"Time to process chunks: {response_time} seconds")

# Extract just the json structured output from a chat completion
structured_output = extract_and_display_output(response)

# Display markdown table
display_json_as_markdown(structured_output)

[{
        "topic": "Definition of Confidential Information",
        "summary": "Doc1 provides a broader definition of Confidential Information, focusing on the nature of the information and the circumstances of disclosure. Doc2 specifically includes developer credentials and communications, with detailed handling requirements.",
        "doc1_context": "Confidential Information shall mean all information that is identified as confidential at the time of disclosure or should be reasonably known to be confidential or proprietary due to the nature of the information disclosed and the circumstances surrounding the disclosure. Each party (as “Receiving Party”) will use the same degree of care that it uses to protect the confidentiality of its own confidential information of like kind (but not less than reasonable care) to (i) not use any Confidential Information of the other party (“Disclosing Party”) for any purpose outside the scope of this Agreement, and (ii) except as otherwise author

| topic | summary | doc1_context | doc2_context |
| --- | --- | --- | --- |
| Definition of Confidential Information | Doc1 provides a broader definition of Confidential Information, focusing on the nature of the information and the circumstances of disclosure. Doc2 specifically includes developer credentials and communications, with detailed handling requirements. | Confidential Information shall mean all information that is identified as confidential at the time of disclosure or should be reasonably known to be confidential or proprietary due to the nature of the information disclosed and the circumstances surrounding the disclosure. Each party (as “Receiving Party”) will use the same degree of care that it uses to protect the confidentiality of its own confidential information of like kind (but not less than reasonable care) to (i) not use any Confidential Information of the other party (“Disclosing Party”) for any purpose outside the scope of this Agreement, and (ii) except as otherwise authorized by the Disclosing Party in writing, limit access to Confidential Information of the Disclosing Party to those of its and its Affiliates’ employees and contractors who need that access for purposes consistent with this Agreement and who have signed confidentiality agreements with the Receiving Party containing protections not materially less protective of the Confidential Information than those herein. If Receiving Party is required by law or court order to disclose Confidential Information, then Receiving Party shall, to the extent legally permitted, provide Disclosing Party with advance written notification and cooperate in any effort to obtain confidential treatment of the Confidential Information. The Receiving Party acknowledges that disclosure of Confidential Information would cause substantial harm for which damages alone would not be a sufficient remedy, and therefore that upon any such violative disclosure by the Receiving Party, the Disclosing Party will be entitled to seek appropriate equitable relief in addition to any other remedies available at law. | Confidential Information includes any materials, communications, and information that are marked confidential or that would normally be considered confidential under the circumstances. Developer credentials (such as passwords, keys, and client IDs) are intended to be used by you and identify your API Client. You will keep your credentials confidential and make reasonable efforts to prevent and discourage other API Clients from using your credentials. Developer credentials may not be embedded in open source projects. Our communications to you and our APIs may contain Google confidential information. If you receive any such information, then you will not disclose it to any third party without Google's prior written consent. Google confidential information does not include information that you independently developed, that was rightfully given to you by a third party without confidentiality obligation, or that becomes public through no fault of your own. You may disclose Google confidential information when compelled to do so by law if you provide us reasonable prior notice, unless a court orders that we not receive notice. |
| Permitted Use & Restrictions | Doc1 allows use for the Customer's benefit with general restrictions, while Doc2 has more specific restrictions, including prohibitions on certain activities and requirements for end-user compliance. | The Service may include software, content, data, or other materials that are owned by persons or entities other than SambaNova and that are provided to Customer on licensee terms that are in addition to and/or different from those contained in this Agreement. Customer is bound by and shall comply with, and is responsible for ensuring that each User is bound by and complies with, all Third-Party Licenses. Any breach by Customer or any of its Users of any Third-Party License is also a breach of this Agreement. | You will require your end users to comply with (and not knowingly enable them to violate) applicable law, regulation, and the Terms. You will comply with all applicable law, regulation, and third party rights (including without limitation laws regarding the import or export of data or software, privacy, and local laws). You will not use the APIs to encourage or promote illegal activity or violation of third party rights. You will not violate any other terms of service with Google (or its affiliates). |
| Data Security | Doc1 requires general security measures and places compliance burden on the Customer, whereas Doc2 provides detailed data protection requirements and privacy policies. | You must implement and use reasonable and appropriate measures to help secure the Service by you and your Users. If you discover any vulnerabilities or breaches, you must promptly inform us and provide us details of such. As a generally available cloud compute service, we will provide the Service in accordance with our obligations under laws applicable to our Service generally, and without regard to your specific and particular use of the Service. If you use the Service to process personal data, or any other specified types of data requiring compliance with any applicable laws or regulations, you represent to us that you have established and maintained any such compliance, and have provided all applicable privacy notices and obtained necessary consents to process such data using the Service, and that you are processing such data using the Service in accordance with all applicable law. You understand and agree that we are not responsible for establishing, providing, obtaining, or maintaining any such compliance, notices and/or consents, and that we may not have done so. | You will use commercially reasonable efforts to protect user information collected by your API Client, including personal data, from unauthorized access or use and will promptly report to your users any unauthorized access or use of such information to the extent required by applicable law. You will comply with (1) all applicable privacy laws and regulations including those applying to personal data and (2) the Google API Services User Data Policy, which governs your use of the APIs when you request access to Google user information. You will provide and adhere to a privacy policy for your API Client that clearly and accurately describes to users of your API Client what user information you collect and how you use and share such information (including for advertising) with Google and third parties. |
